In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np
import os
from training_audio_model import *
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())
# Chargement des données
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=True)

data[:3]

C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


True


,ipu_id,speaker,start_ipu,stop_ipu,text_ipu,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,start_words,stop_words,duration,text_words,request_after_word,turn_after_word,is_ipu_end,dyad
0,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,4.54,4.72,0.18,tu,False,False,False,transcr\AAOR
1,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,4.72,4.84,0.12,as,False,False,True,transcr\AAOR
2,1,OR,5.14,5.825,mh ouais si tu veux,True,True,False,NaN,False,True,5.14,5.44,0.30,mh,True,False,False,transcr\AAOR


In [19]:

df = pd.DataFrame(data)
from utils import *
y=create_y(df)
print(len(y))
print(y.count(1))

36451
9508


In [3]:

audio_files_path = 'paco-cheese/audio/2_channels/'


audio_segments = extract_audio_segments(data,audio_files_path)
audio_segments

 ...]

In [4]:
print(len(audio_segments))
print(len(y))


110544
110544


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier



X = np.array([extract_features(segment) for segment in audio_segments])


C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1600
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1280
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1920
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=13

In [16]:
dataset = AudioDataset(X, y)
train_size=int(0.8*len(dataset))
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Fixez le seed pour la reproductibilité
seed = 42

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Créez les objets AudioDataset pour l'entraînement et le test
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

# Créez les DataLoaders pour ces ensembles de données
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



ValueError: Found input variables with inconsistent numbers of samples: [110544, 36451]

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = AudioCNN().to(device)



criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Paramètres d'entraînement
num_epochs = 10
class_weights = torch.tensor([1.0, 4.0], device=device)

loss_fn = nn.CrossEntropyLoss(weight=class_weights)

#entraînement
for epoch in range(num_epochs):

    model.train()
    
    for inputs, labels in train_loader:
    

   
        inputs, labels = inputs.to(device), labels.to(device)
       

        optimizer.zero_grad()


        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        loss.backward()
        optimizer.step()

    # Évaluation
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
          
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%')
    torch.save(model, 'modele/model_audio')

Using device: cuda
Epoch [1/10], Loss: 0.3235, Accuracy: 86.27%
Epoch [2/10], Loss: 0.4782, Accuracy: 85.80%
Epoch [3/10], Loss: 0.4750, Accuracy: 87.58%
Epoch [4/10], Loss: 0.4155, Accuracy: 86.56%
Epoch [5/10], Loss: 0.4587, Accuracy: 86.26%
Epoch [6/10], Loss: 0.6009, Accuracy: 82.62%
Epoch [7/10], Loss: 0.3189, Accuracy: 81.62%
Epoch [8/10], Loss: 0.6329, Accuracy: 83.17%
Epoch [9/10], Loss: 0.3130, Accuracy: 85.77%
Epoch [10/10], Loss: 0.6097, Accuracy: 84.65%


In [12]:
print(len(y))
print(len(y_test))

110544
22109


In [13]:
def predition_model_audio(model,dataset,device,proba=True):
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataset:
            inputs, labels = inputs.to(device), labels.to(device)

            preds = model(inputs)
            if not proba:
                _, preds = torch.max(preds.data, dim=1)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    return all_preds,all_labels
all_preds_audio,all_labels=predition_model_audio(model,test_loader,device,proba=False)
f1 = f1_score(all_labels, all_preds_audio)#, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds_audio)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')

Test F1 Score: 0.41903457719958925
Confusion Matrix:
[[17491  2320]
 [ 1074  1224]]
Nombre d'éléments de classe 0 détectés : 17491 sur 19811
Nombre d'éléments de classe 1 détectés : 1224 sur 2298
